# Class Explanation (クラスに関する説明)

## phase2

### Kalman Filter (kalman.py)

In [1]:
'''Kalman_Filter(observation = None, initial_mean = None, initial_covariance = None, transition_matrices = None, observation_matrices = None, transition_covariance = None, observation_covariance = None, transition_noise_matrices = None, transition_offsets = None, observation_offsets = None, em_vars=['transition_covariance', 'observation_covariance', 'initial_mean', 'initial_covariance'], em_dics = {}, n_dim_sys = None, n_dim_obs = None)'''

"Kalman_Filter(observation = None, initial_mean = None, initial_covariance = None, transition_matrices = None, observation_matrices = None, transition_covariance = None, observation_covariance = None, transition_noise_matrices = None, transition_offsets = None, observation_offsets = None, em_vars=['transition_covariance', 'observation_covariance', 'initial_mean', 'initial_covariance'], em_dics = {}, n_dim_sys = None, n_dim_obs = None)"

状態方程式，観測方程式は以下で与えられている．ただし，オフセット$\mathbf{b}_t,\mathbf{d}_t$，状態ノイズ変換行列$G_t$は抜いても良い．

$$
\begin{align*}
&\mathbf{x}_{t+1}=F_t\mathbf{x}_t+\mathbf{b}_t+G_t\mathbf{v}_t,\ \mathbf{v}_t\sim N(0,Q_t)\\
&\mathbf{y}_t=H_t\mathbf{x}_t+\mathbf{d}_t+\mathbf{w}_t,\ \mathbf{w}_t\sim N(0,R_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|initial_covariance |$V[\mathbf{x}_0]$ |np.eye |初期状態の共分散
|transition_matrices |$\{F_t\}$ |np.eye |状態遷移行列
|observation_matrices |$\{H_t\}$ |np.eye |観測行列
|transition_covariance |$\{Q_t\}$ |np.eye |状態ノイズ共分散行列
|observation_covariance |$\{R_t\}$ |np.eye |観測ノイズ共分散行列
|transition_noise_matrices |$\{G_t\}$ |np.eye |状態ノイズ変換行列
|transition_offsets |$\mathbf{b}_t$ |np.zeros |状態遷移オフセット
|observation_offsets |$\mathbf{d}_t$ |np.zeros |観測オフセット
|em_vars | |$Q,R,E[\mathbf{x}_0],V[\mathbf{x}_0]$ |EMアルゴリズムで最適化したい変数名を与えるリスト型変数
|em_dics | |{} |EMアルゴリズムでパラメータを最適化する際に行列・ベクトルの特定の値を固定したい際に用いる辞書型変数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元

格納されている関数

|関数名 |説明
|:--|:--
|filter |一気先予測・フィルターを行う関数
|RTS_smooth |RTS平滑化を行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数
|get_RTS_smoothed_value(dim=None) |第dim次元の平滑化値を得る関数
|em(n_iter=10) |EMアルゴリズムを用いてパラメータ(em_vars)をn_iter回イテレーションを回し最適化する関数

em_vars について
- EMアルゴリズムで最適化したい変数をリストで渡す．
    - 例えば，em_vars = ['transition_covariance', 'initial_mean'] と与える
- 最適化可能対象は次のとおりである．
    - 'transition_matrices'
    - 'observation_matrices'
    - 'transition_covariance'
    - 'observation_covariance'
    - 'initial_mean'
    - 'initial_covariance'
    - 'transition_offsets'
    - 'observation_offsets'

em_dics について
- EMアルゴリズム適用時に，行列内，ベクトル内の特定の要素を最適化したい時に用いる．
    - 例えば，状態遷移行列の対角成分だけを固定したいような場合が考えられる．
        - 状態遷移行列がpxp行列の場合，em_dics = {'transition_matrices' : [list(range(p)), list(range(p))]} とすれば良い．
- 辞書の key として，固定したい対象の変数の名前，value として固定したい要素番号を入れて用いる．
    - 2x2状態遷移行列の対角成分を固定したい場合
        - em_dics = {'transition_matrices' : [[0, 1], [0, 1]]}
        - [0,0], [1,1] ではなく，二つのリストを用いて要素の場所を指定していることに注意されたい．
    - 2x2状態遷移行列の非対角成分[0,1]を固定したい場合
        - em_dics = {'transition_matrices' : [[0], [1]]}
    - 5次元状態オフセットベクトルの第0,3成分を固定したい場合
        - em_dics = {'transition_offsets' : [0, 3]}
        - ベクトルに関しては直観的に分かりやすい仕様．
    - pxp状態遷移行列，kxk観測行列の対角成分を固定したい場合
        - em_dics = {'transition_covariance' : [list(range(p)), list(range(p))],
            'observation_covariance' : [list(range(k)), list(range(k))]}
        - 複数のパラメータで固定する成分がある場合は，それぞれkeyを作れば良い
- 使用例は，phase2/180313_Kalman_Filter_for_class_make.ipynb を参照にして欲しい．
    - クラス作りながらメモ書きしたり実験したりする用のため，コードとしてはまとまってないですが．

今後，追加したい機能

|機能 |説明
|:--|:--
|状態方程式の拡張 |$x_{t+1}=F_tx_t+\Gamma_tu_t+G_tv_t$に状態方程式を拡張したい．隠れ状態$x_t$に関する遷移を考えているが，既知変数$u_t$を含めた形への拡張．
|EM Algorithmの時変性 |現状は時不変な$F,H,Q,R$の最適化を行なっているが，時変な$F_t,H_t,Q_t,R_t$への拡張．

### Ensemble Kalman Filter (ensemble.py)

In [4]:
'''Ensemble_Kalman_Filter(self, observation = None, transition_functions = None, observation_matrices = None, initial_mean = None, transition_noise = None, observation_covariance = None, n_particle = 100, n_dim_sys = None, n_dim_obs = None, seed = 71)'''

'Ensemble_Kalman_Filter(self, observation = None, transition_functions = None, observation_matrices = None, initial_mean = None, transition_noise = None, observation_covariance = None, n_particle = 100, n_dim_sys = None, n_dim_obs = None, seed = 71)'

状態方程式，観測方程式は次のように与えられている．

$$
\begin{align*}
&\mathbf{x}_{t+1}=\mathbf{f}_t(\mathbf{x}_t)+\mathbf{v}_t,\ \mathbf{v}_t\sim p(\mathbf{v}_t)\\
&\mathbf{y}_t=H_t\mathbf{x}_t+\mathbf{w}_t,\ \mathbf{w}_t\sim N(0,R_t)
\end{align*}
$$

必要な変数

|変数名  |対応文字  |デフォルト値 |説明
|:--|:-:|:-:|:--|
|observation  |$\{\mathbf{y}_t\}$ |  |観測変量
|initial_mean  |$E[\mathbf{x}_0]$ |np.zeros |初期状態の平均
|transition_functions |$\{\mathbf{f}_t\}$ |lambda x,v:x+v |状態遷移関数
|observation_matrices |$\{H_t\}$ |np.eye |観測行列
|transition_noise |$\{p(\mathbf{v}_t)\}$ |(np.multivariatenormal, [np.zeros,np.eye]) |パラメトリック状態ノイズ
|observation_covariance |$\{R_t\}$ |np.eye |観測ノイズ共分散行列
|n_particle | |100 |アンサンブルメンバー数
|n_dim_sys | | |状態変数の次元
|n_dim_obs| | |観測変数の次元
|seed | |71 |乱数発生シード

格納されている関数

|関数名 |説明
|:--|:--
|filter |一気先予測・フィルターを行う関数
|smooth(lag=10) |lag間隔で固定ラグ平滑化を行う関数
|get_predicted_value(dim=None) |第dim次元の一気先予測値(背景値)を得る関数
|get_filtered_value(dim=None) |第dim次元のフィルター値(解析値)を得る関数
|get_smoothed_value(dim=None) |第dim次元の平滑化値を得る関数

今後，編集・追加したいこと

|内容 |説明
|:--|:--
|フィルター更新式の高速化 |Evensen の計算法でオーダーを下げる
|平滑化の動作確認 |Smoothing の結果の妥当性の検証
|使える乱数の拡張 |現状，numpy で使用可能な乱数を入れる構造になっているが，自己正規化サンプリングや逐次サンプリング等が使えるような拡張の検討